# Use Embedding API
Note: To facilitate embedding API calls, the key should be filled in the .env file under llm_universe, and the code will automatically read and load the environment variables.
## 1. Use OpenAI API
GPT has a packaged interface, we can simply package it. Currently, there are three GPT embedding modes, and the performance is as follows:
|Model | Pages per dollar | [MTEB](https://github.com/embeddings-benchmark/mteb) score | [MIRACL](https://github.com/project-miracl/miracl) score|
| --- | --- | --- | --- |
|text-embedding-3-large|9,615|64.6|54.9|
|text-embedding-3-small|62,500|62.3|44.0|
|text-embedding-ada-002|12,500|61.0|31.4|
* MTEB score is the average score of eight tasks such as classification, clustering, and pairing of the embedding model.
* MIRACL score is the average score of the embedding model inAverage score on the retrieval task. 

From the above three embedding models, we can see that `text-embedding-3-large` has the best performance and the most expensive price. We can use it when the application we build needs better performance and the cost is sufficient; `text-embedding-3-small` has better performance and price. We can choose this model when our budget is limited; and `text-embedding-ada-002` is the previous generation model of OpenAI. It is inferior to the previous two in terms of performance and price, so it is not recommended.

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv


# Read local/project environment variables.
# find_dotenv() finds and locates the path of the .env file
# load_dotenv() reads the .env file and loads the environment variables in it into the current running environment
# If you set a global environment variable, this line of code will have no effect.
_ = load_dotenv(find_dotenv())

# If you need to access through the proxy port, you need to configure as follows
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'

def openai_embedding(text: str, model: str=None):
# Get the environment variable OPENAI_API_KEY
    api_key=os.environ['OPENAI_API_KEY']
    client = OpenAI(api_key=api_key)

# embedding model: 'text-embedding-3-small', 'text-embedding-3-large', 'text-embedding-ada-002'
    if model == None:
        model="text-embedding-3-small"

    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response

response = openai_embedding(text='要生成 embedding 的输入文本，字符串形式。')

The data returned by the API is in `json` format. In addition to the `object` vector type, there are also data `data` for storing data, the embedding model model `model`, and the usage of this token `usage`, as shown below:
```json
{
"object": "list",
"data": [
{
"object": "embedding",
"index": 0,
"embedding": [
-0.006929283495992422,
... (omitted)
-4.547132266452536e-05,
],
}
],
"model": "text-embedding-3-small",
"usage": {
"prompt_tokens": 5,
"total_tokens": 5
}
}
```
We can call the response object to get the type of embedding.

In [2]:
print(f'返回的embedding类型为：{response.object}')

返回的embedding类型为：list


The embedding is stored in data, and we can check the length of the embedding and the generated embedding.

In [3]:
print(f'embedding长度为：{len(response.data[0].embedding)}')
print(f'embedding（前10）为：{response.data[0].embedding[:10]}')

embedding长度为：1536
embedding（前10）为：[0.03884002938866615, 0.013516489416360855, -0.0024250170681625605, -0.01655769906938076, 0.024130908772349358, -0.017382603138685226, 0.04206013306975365, 0.011498954147100449, -0.028245486319065094, -0.00674333656206727]


We can also view the model and token usage of this embedding.

In [4]:
print(f'本次embedding model为：{response.model}')
print(f'本次token使用情况为：{response.usage}')

本次embedding model为：text-embedding-3-small
本次token使用情况为：Usage(prompt_tokens=12, total_tokens=12)


## 2. Use Wenxin Qianfan API
Embedding-V1 is a text representation model based on Baidu Wenxin large model technology. Access token is the credential for calling the interface. When using Embedding-V1, you should first obtain Access token with API Key and Secret Key, and then use Access token to call the interface to embed text. At the same time, Qianfan large model platform also supports embedding models such as bge-large-zh.

In [5]:
import requests
import json

def wenxin_embedding(text: str):
# Get environment variables wenxin_api_key, wenxin_secret_key
    api_key = os.environ['QIANFAN_AK']
    secret_key = os.environ['QIANFAN_SK']

# Use API Key and Secret Key to obtain Access token from https://aip.baidubce.com/oauth/2.0/token
    url = "https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={0}&client_secret={1}".format(api_key, secret_key)
    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    
# Embed text using the obtained Access token
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/embeddings/embedding-v1?access_token=" + str(response.json().get("access_token"))
    input = []
    input.append(text)
    payload = json.dumps({
        "input": input
    })
    headers = {
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return json.loads(response.text)
# text should be List(string)
text = "要生成 embedding 的输入文本，字符串形式。"
response = wenxin_embedding(text=text)

In Embedding-V1, each embedding has a separate id and a timestamp to record the time of embedding.

In [6]:
print('本次embedding id为：{}'.format(response['id']))
print('本次embedding产生时间戳为：{}'.format(response['created']))

本次embedding id为：as-hvbgfuk29u
本次embedding产生时间戳为：1711435238


Similarly, we can also get the embedding type and embedding from the response.

In [7]:
print('返回的embedding类型为:{}'.format(response['object']))
print('embedding长度为：{}'.format(len(response['data'][0]['embedding'])))
print('embedding（前10）为：{}'.format(response['data'][0]['embedding'][:10]))

返回的embedding类型为:embedding_list
embedding长度为：384
embedding（前10）为：[0.060567744076251984, 0.020958080887794495, 0.053234219551086426, 0.02243831567466259, -0.024505289271473885, -0.09820500761270523, 0.04375714063644409, -0.009092536754906178, -0.020122773945331573, 0.015808865427970886]


## 3. Use iFlytek Spark API

### Not yet open

## 4. Use Zhipu API
Zhipu has a packaged SDK, we can just call it.

In [9]:
from zhipuai import ZhipuAI
def zhipu_embedding(text: str):

    api_key = os.environ['ZHIPUAI_API_KEY']
    client = ZhipuAI(api_key=api_key)
    response = client.embeddings.create(
        model="embedding-2",
        input=text,
    )
    return response

text = '要生成 embedding 的输入文本，字符串形式。'
response = zhipu_embedding(text=text)

The response is of type `zhipuai.types.embeddings.EmbeddingsResponded`. We can call `object`, `data`, `model`, and `usage` to view the embedding type, embedding, embedding model, and usage of the response.

In [10]:
print(f'response类型为：{type(response)}')
print(f'embedding类型为：{response.object}')
print(f'生成embedding的model为：{response.model}')
print(f'生成的embedding长度为：{len(response.data[0].embedding)}')
print(f'embedding（前10）为: {response.data[0].embedding[:10]}')

response类型为：<class 'zhipuai.types.embeddings.EmbeddingsResponded'>
embedding类型为：list
生成embedding的model为：embedding-2
生成的embedding长度为：1024
embedding（前10）为: [0.017892399802803993, 0.0644201710820198, -0.009342825971543789, 0.02707476168870926, 0.004067837726324797, -0.05597858875989914, -0.04223804175853729, -0.03003198653459549, -0.016357755288481712, 0.06777040660381317]
